In [3]:
!pip3 install numpy pandas sklearn arff

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for arff: filename=arff-0.9-py3-none-any.whl size=4970 sha256=1e6bd084063a953659777e06548414da476585f0da89653094cabd454812e26c
  Stored in directory: /Users/victorpires/Library/Caches/pip/wheels/5e/d6/66/a25682c020fb563800fea1a06de58e4684243efd68cb83db83
Successfully built arff
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import re
import arff
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
def weka_tokenizer(doc):
    # Expressão regular usada na tokenização 
    delimiters_regexp = re.compile("[ |\n|\f|\r|\t|.|,|;|:|'|\"|(|)|?|!]")
    return list(filter(None, delimiters_regexp.split(doc)))

In [10]:
def initDataset(fileName, vocabularyFile):
    fileExtension = fileName.split('.')
    if (fileExtension[1] == 'csv'):
        df = pd.read_csv(fileName)
    else:
        data = arff.load(fileName)
        df = pd.DataFrame(data, columns=['token', 'loc', 'abstract_keyword', 'assert_keyword', 'boolean_keyword', 'break_keyword', 'byte_keyword', 'case_keyword', 'catch_keyword', 'char_keyword', 'class_keyword', 'continue_keyword', 'default_keyword', 'do_keyword', 'double_keyword', 'else_keyword', 'enum_keyword', 'exports_keyword', 'extends_keyword', 'final_keyword', 'finally_keyword', 'float_keyword', 'for_keyword', 'if_keyword', 'implements_keyword', 'import_keyword', 'instanceof_keyword', 'int_keyword', 'interface_keyword', 'long_keyword', 'modules_keyword', 'native_keyword', 'new_keyword', 'package_keyword', 'private_keyword', 'protected_keyword', 'public_keyword', 'requires_keyword', 'return_keyword', 'short_keyword', 'static_keyword', 'strictfp_keyword', 'super_keyword', 'switch_keyword', 'synchronized_keyword', 'this_keyword', 'throw_keyword', 'throws_keyword', 'transient_keyword', 'try_keyword', 'void_keyword', 'volatile_keyword', 'while_keyword', 'true_keyword', 'null_keyword', 'false_keyword', 'const_keyword', 'goto_keyword', 'keywordcount', 'klass'])

    y = df['klass']

    if (fileExtension[1] == 'csv'):
        vectorizer = CountVectorizer(analyzer='word', max_features=1500, tokenizer=weka_tokenizer) 
    else:
        vectorizer = CountVectorizer(analyzer='word', max_features=1500, tokenizer=weka_tokenizer) 

    bowToken = vectorizer.fit_transform(df['token'])

    bowData = pd.DataFrame(bowToken.toarray(), columns=vectorizer.get_feature_names_out())

    df.drop('token', axis=1, inplace=True)
    df = df.join(bowData)
    df.drop('klass', axis=1, inplace=True)
    if (fileExtension[1] == 'csv'):
        df.drop('project', axis=1, inplace=True)
    x = df

    return x, y

In [52]:
# x, y = initDataset('../datasets/flakies_rq_21.csv', '')
# x, y = initDataset('../datasets/flakies_rq_22.csv', '')
x, y = initDataset('../datasets/MSR4FlakinessOriginal.arff', '')

skf = StratifiedKFold(n_splits=10)

for train_index, test_index in skf.split(x,y):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

/Users/victorpires/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
